In [ ]:
#importing library
#for accessing data
import os
import glob

#for image pre-processing
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#for model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2
from keras.layers.core import Activation, Flatten, Dense

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#info on dataset
datasetPath="/content/drive/MyDrive/Kaggle/plantDisease"
train_dir="/content/drive/MyDrive/Kaggle/plantDisease/train"
val_dir="/content/drive/MyDrive/Kaggle/plantDisease/val"
test_dir="/content/drive/MyDrive/Kaggle/plantDisease/test"

totalClasses=16

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
val_datagen=ImageDataGenerator(rescale=1./255)

# set height and width and color of input image.
img_width,img_height =256,256
input_shape=(img_width,img_height,3)
batch_size =32

train_generator=train_datagen.flow_from_directory(train_dir,target_size=(img_width,img_height),batch_size=batch_size)
val_generator=val_datagen.flow_from_directory(val_dir,shuffle=True,target_size=(img_width,img_height),batch_size=batch_size)

Found 15994 images belonging to 16 classes.
Found 1997 images belonging to 16 classes.


In [ ]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('/content/drive/MyDrive/ColabNotebooks/PDDCS/TrainedModels/labels.txt','w') as f:
  f.write(labels)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___Northern_Leaf_Blight': 9, 'Corn_(maize)___healthy': 10, 'Grape___Black_rot': 11, 'Grape___Esca_(Black_Measles)': 12, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 13, 'Grape___healthy': 14, 'Orange___Haunglongbing_(Citrus_greening)': 15}


In [ ]:
# Xception model

model=keras.Sequential()

model.add(MobileNetV2(include_top=False,pooling='max',weights='imagenet'))

model.add(Dense(totalClasses,activation='softmax'))

model.layers[0].trainable=False 

model.summary()

9412608/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1280)              2257984   
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 16)                20496     
Total params: 2,278,480
Trainable params: 20,496
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# checkpoint

checkpointPath = '/content/drive/MyDrive/ColabNotebooks/PDDCS/TrainedModels/trainingStopped/mobileNetV2/'

modelCheckpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpointPath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

In [ ]:
opt=keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.load_weights(checkpointPath)

train=model.fit(train_generator,epochs=33,steps_per_epoch=train_generator.samples//batch_size,validation_data=val_generator,validation_steps=val_generator.samples//batch_size,verbose=1,callbacks=[modelCheckpointCallback])

Epoch 1/33
 42/499 [=>............................] - ETA: 1:00:51 - loss: 0.0735 - accuracy: 0.9851

KeyboardInterrupt: ignored

In [ ]:
# Save model

saveDir = '/content/drive/MyDrive/ColabNotebooks/PDDCS/TrainedModels/mobileNetV2'

tf.saved_model.save(model,saveDir)

converter = tf.lite.TFLiteConverter.from_saved_model(saveDir)

tfliteModel = converter.convert()

with open('/content/drive/MyDrive/ColabNotebooks/PDDCS/TrainedModels/model.tflite','wb') as f:
  f.write(tfliteModel)

In [ ]:
# plotting graph for Training and Validation loss

loss_train = train.history['loss']
loss_val = train.history['val_loss']
epochs = range(1,14)
plt.plot(epochs,loss_train, 'g', label='Training loss')
plt.plot(epochs,loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# plotting graph for Training and Validation accuracy

train_accuracy = train.history['accuracy']
val_accuracy = train.history['val_accuracy']
epochs = range(1,11)
plt.plot(epochs,train_accuracy,'g',label='Train accuracy')
plt.plot(epochs,val_accuracy,'b',label='Val accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()